# 3. Processamento/Tratamento de Dados

### Instalar/Importar bibliotecas

In [1]:
# ! pip install xlrd
# ! pip install openpyxl
# ! pip install pydantic
# ! pip install joblib

In [2]:
import numpy as np
import pandas as pd
import joblib

### Carregar dados resultado da aquisicao

In [3]:
def carregar_dados_aquisicao():
    df = joblib.load("../out/result_aquisicao.joblib")
    print("arquivo joblib carregado com sucesso")

    return df

## Criacao de rotulos - coluna target

### Criar coluna resultado baseado em cotacao e IBOV

- cotacao e Ibov, considerar valores do proximo trimestre 
- Considerar comprar (1): variacao da cotacao acima da variacao do IBOV
- Considerar Vender (0): variacao da cotacao de 2% abaixo da variacao do IBOV

In [4]:
def criar_coluna_decisao(dfs_in): 
    
    dfs = dfs_in.copy()

    for code in dfs.keys():
        df = dfs[code]
        
        df = df.sort_index()

        df["cotacao_var"] = df["Adj Close"].shift(-1) / df["Adj Close"] - 1
        df["IBOV_var"] = df["IBOV"].shift(-1) / df["IBOV"] - 1
        df["resultado"] = df["cotacao_var"] - df["IBOV_var"]

        condicoes = [
            (df["resultado"] > 0 ), 
            (df["resultado"] < -0.02)
        ]
        valores = [1, 0]

        df["decisao"] = np.select(condicoes, valores)

        df = df.drop(["Adj Close", "cotacao_var", "IBOV", "IBOV_var", "resultado"], axis=1)

        dfs[code] = df

    print("coluna decisao/target criada com sucesso")

    return dfs

### Transformar valores em porcentagem

Para cada campo, calcular porcentagem em relacao ao proximo registro (trimestre), menos para o campo de decisao.

In [5]:
def transformar_indic_em_porcentagem_delta(dfs_in):
    dfs = dfs_in.copy()

    for code in dfs.keys():
        fundamento = dfs[code]
        fundamento.sort_index()
        for coluna in fundamento.columns:
            if coluna != "decisao":
                condicoes = [
                    (fundamento[coluna].shift(1) > 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] > 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] > 0),
                    (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] == 0),
                ]
                valores = [
                    -1,
                    1,
                    (abs(fundamento[coluna].shift(1)) - abs(fundamento[coluna]))
                                / abs(fundamento[coluna].shift(1)),
                    1,
                    -1,
                    1,
                ]
                fundamento[coluna] = (
                    np.select(condicoes, valores, default=fundamento[coluna]
                              / fundamento[coluna].shift(1) - 1)
                )
        dfs[code] = fundamento
    return dfs

### Juntar dataframes

Finalmente junta-se os dataframes contidos no dicionario, que representam cada empresa em um unico dataframe.

In [6]:
def juntar_dataframes(dfs_in):
    copia_fund = dfs_in.copy()
    bd = pd.DataFrame()
    for code in copia_fund:
        copia_fund[code] = copia_fund[code][1:-1]
        copia_fund[code] = copia_fund[code].reset_index(drop=True)
        bd = bd.append(copia_fund[code])
    bd.reset_index(drop=True, inplace=True)
    print(f"Dataframe unico criado com sucesso. Registros: {len(bd)}")
    return bd

### Padronizacao dos nomes das colunas

Renomear colunas removendo espacos, acentos e simbolos especiais.

In [7]:
def rename_columns(df):
    df = df.copy()
    columns_rename={
        "Ativo Total": "ativo_total", 
        "Ativo Circulante": "ativo_circ",
        "Ativo Realizável a Longo Prazo": "ativo_permanente",
        "Passivo Circulante": "passivo_circ", 
        "Passivo Não Circulante": "passivo_nao_circ",
        "Patrimônio Líquido": "patrimonio_liq", 
        "Receita Líquida de Vendas e/ou Serviços": "receita_liq", 
        "Custo de Bens e/ou Serviços Vendidos": "custo_bens_servicos", 
        "Lucro/Prejuízo do Período": "lucro_op", 
        "Outros Ativos Circulantes": "outros_ativos_circ", 
        "Caixa e Equivalentes de Caixa": "caixa_equiv_caixa",
        "Outros Ativos Não Circulantes": "outros_ativos_nao_circ", 
        "Imobilizado": "imobilizado", 
        "Intangível": "intangivel",
        "Passivo Total": "passivo_total", 
        "Capital Social Realizado": "capital_social",
        "Resultado Bruto": "resultado_bruto",
        "Despesas Gerais e Administrativas": "despesas_gerais", 
        "Financeiras": "financeiras",
        "Resultado Antes Tributação/Participações": "resultado_antes_trib"}

    df.rename(columns=columns_rename, inplace=True)
    df.iloc[:,:-1] = df.iloc[:,:-1].astype(float)
    return df

### Salvar dataframe em joblib

Armazena o resultado do notebook para ser processado pela proxima etapa.

In [8]:
def salvar_df_in_joblib(df, file_name: str = "../out/tratamento_target.joblib"):

    joblib.dump(df, file_name)

    print("arquivo joblib de df decisao criado com sucesso")

# Execucao

In [12]:
def main():

    dfs = carregar_dados_aquisicao()

    dfs = criar_coluna_decisao(dfs)

    dfs = transformar_indic_em_porcentagem_delta(dfs)

    bd = juntar_dataframes(dfs)

    bd = rename_columns(bd)

    # salvar em arquivo joblib
    salvar_df_in_joblib(bd, file_name="../out/trat_target_col_ibov.joblib")
    
    return bd
    
bd = main()

arquivo joblib carregado com sucesso
coluna decisao/target criada com sucesso
Dataframe unico criado com sucesso. Registros: 1852


In [10]:
bd.head()

,ativo_total,ativo_circ,caixa_equiv_caixa,outros_ativos_circ,ativo_permanente,outros_ativos_nao_circ,imobilizado,intangivel,passivo_total,passivo_circ,...,patrimonio_liq,capital_social,receita_liq,custo_bens_servicos,resultado_bruto,despesas_gerais,financeiras,resultado_antes_trib,lucro_op,decisao
0,0.006519,0.016469,0.567359,0.073416,0.053858,0.108915,-0.012565,-0.020877,0.006519,-0.028162,...,0.018187,0.072637,-0.009957,0.025893,0.084093,-0.178706,0.397405,0.784128,0.714242,1
1,0.005025,0.558803,-0.495118,21.051236,-0.288000,-0.292028,-0.426257,-0.340684,0.005025,0.291278,...,-0.057218,0.000000,-0.306352,0.266717,-0.516536,0.322396,-1.348700,-6.964708,-4.169923,0
2,-0.306583,-0.499544,0.592010,-0.954322,-0.005642,0.004897,0.026113,0.023500,-0.306583,-0.582626,...,-0.212056,0.000000,0.109726,-0.099746,0.190001,0.103272,0.430909,0.649179,0.216400,0
3,-0.006799,-0.076635,-0.334499,0.088899,0.068368,0.035394,0.045404,0.044928,-0.006799,-0.083899,...,-0.036367,0.000000,-0.407359,0.376221,-0.638821,0.330099,0.284087,0.540194,0.746589,1
4,0.018784,0.042057,0.613017,-0.242792,0.075156,0.128516,-0.018718,-0.026987,0.018784,0.018482,...,-0.031893,0.000000,0.633929,-0.533540,1.922709,-0.603944,-0.125983,-0.245369,-0.156249,1
